# Medical Assistant Powered by the Azure OpenAI Service

In this sample we will explore how to use the Azure OpenAI Service to build a medical assistant. 

The doctor inputs Medical Shorthand (`shorthand`) and then we kick off the following stages:
1. Convert the Medical Shorthand to a Clinical Note. (`clinical_note`)
1. Convert the Medical Shorthand to a Structured Clinical Note. (`clinical_note_structured`) 
    1. Convert the Clinical Note to a Structured Clinical Note. (`clinical_note_structured`) - *variation*
1. Analyse the Clinical Note to check if the doctor has missed anything? (`feedback`)

## Azure OpenAI Service Setup

We will kick things off by:
- Importing the required libraries 
- Setting up the Azure OpenAI Service

Note: You must to input your:
- Azure OpenAI Service Key.
- Azure OpenAI Service Endpoint.
- Azure OpenAI Service Model Deployment Name.

In [ ]:
%pip install openai
import openai
openai.api_version = '2022-12-01'
openai.api_base = 'https://YOUR-ENDPOINT.openai.azure.com/' # Please add your endpoint here
openai.api_type = 'azure'
openai.api_key = '********************************'         # Please add your api key here
deployment_name = "YOUR-DEPLOYMENT-NAME"                    # Please add your deployment id here

## Scenario Setup

Here we will setup the scenario by defining the Clinical Shorthand (`shorthand`) that we will be using for this sample.

Note: You can change the Clinical Shorthand to anything you like.

In [44]:
# This is a sample medical shorthand note written by a doctor after seeing a patient.
shorthand = """
John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not
"""

## Process Medical Shorthand

Here we will use the Azure OpenAI Service to process the Medical Shorthand (`shorthand`).

### Step 1: Convert the Medical Shorthand to a Clinical Note

- **Input**: Medical Shorthand (`shorthand`)
- **Output**: Clinical Note (`clinical_note`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
******THE-MEDICAL-SHORTHAND-GOES-HERE******
<|im_end|>
<|im_start|>assistant
Clinical Note:
```

In [50]:
prompt_step_1 = """
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
""" + shorthand + """
<|im_end|>
<|im_start|>assistant
Clinical Note:
"""
completion_step_1 = openai.Completion.create(
    deployment_id=deployment_name,
    prompt=prompt_step_1 , 
    stop="<|im_end|>", 
    temperature=0.3,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0)

clinical_note = completion_step_1['choices'][0]['text']

print("Medical Shorthand:")
print(shorthand)
print("Clinical Note:")
print(clinical_note)

Medical Shorthand:

John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not

Clinical Note:
Patient is John Brown, a 73-year-old male. He presents with complaints of chronic progressive onset of exertional dyspnea (CPOE) and shortness of breath on exertion (SOBOE) for the past 3 months. His past medical history (PMHx) includes a ST-elevation myocardial infarction (STEMI) in 2002, which was treated with percutaneous coronary intervention (PCI) of the left anterior descending artery (LAD) with a drug-eluting stent (HREF). He also has chronic kidney disease (CKD), a history of cerebrovascular accident (CVA), and atrial fibrillation (AF) for which he is currently taking a non-vitamin K antagonist oral anticoagulant (NOAC) rivaroxaban. His current m

### Step 2 (A): Convert the Medical Shorthand to a Structured Clinical Note.

- **Input**: Medical Shorthand (`shorthand`)
- **Output**: Clinical Note (`clinical_note_structured`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a structured clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
******THE-MEDICAL-SHORTHAND-GOES-HERE******
<|im_end|>
<|im_start|>assistant
Clinical Note:
```

In [51]:
prompt_step_2 = """
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a structured clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
""" + shorthand + """
<|im_end|>
<|im_start|>assistant
"""
completion_step_2 = openai.Completion.create(
    deployment_id=deployment_name,
    prompt=prompt_step_2 , 
    stop="<|im_end|>", 
    temperature=0.3,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0)

clinical_note_structured = completion_step_2['choices'][0]['text']

print("Medical Shorthand:")
print(shorthand)
print("Clinical Note:")
print(clinical_note_structured)

Medical Shorthand:

John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not

Clinical Note:
Patient: John Brown, 73-year-old male

Symptoms: Chronic progressive exertional dyspnea (CPOE) and shortness of breath on exertion (SOBOE) for the past 3 months, typical for the patient.

Past Medical History:
1. ST-elevation myocardial infarction (STEMI) in 2002.
2. Percutaneous coronary intervention (PCI) for left anterior descending artery (LAD) with a drug-eluting stent.
3. Chronic kidney disease (CKD).
4. Cerebrovascular accident (CVA).
5. Atrial fibrillation (AF) on a novel oral anticoagulant (NOAC).

Medications:
1. Bisoprolol (Bicor).
2. Aspirin.
3. Statin.
4. Rivaroxaban.
5. Sacubitril/valsartan (Entresto).

Physical Examination:
1. Well-appeari

### Step 2 (B): Convert the Clinical Note to a Structured Clinical Note.

- **Input**: Medical Shorthand (`clinical_note`)
- **Output**: Clinical Note (`clinical_note_structured`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant.

Users will paste in a clinical note. You will respond by converting this to a structured clinical note.
<|im_end|>
<|im_start|>user
******THE-LONG-FORM-CLINICAL-NOTE-GOES-HERE******
<|im_end|>
<|im_start|>assistant
Clinical Note:
```


In [52]:
prompt_step_2_b = """
<|im_start|>system
You are an medical assistant. 

Users will paste in a clinical note. You will respond by converting this to a structured clinical note.
<|im_end|>
<|im_start|>user
""" + clinical_note + """
<|im_end|>
<|im_start|>assistant
Clinical Note:
"""

completion_step_2_b = openai.Completion.create(
    deployment_id=deployment_name,
    prompt=prompt_step_2_b , 
    stop="<|im_end|>", 
    temperature=0.3,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0)

clinical_note_structured_b = completion_step_2_b['choices'][0]['text']

print("Long-Form Clinical Note:")
print(clinical_note)
print("\n\n")
print("Structured Clinical Note:")
print(clinical_note_structured_b)

Long-Form Clinical Note:
Patient is John Brown, a 73-year-old male. He presents with complaints of chronic progressive onset of exertional dyspnea (CPOE) and shortness of breath on exertion (SOBOE) for the past 3 months. His past medical history (PMHx) includes a ST-elevation myocardial infarction (STEMI) in 2002, which was treated with percutaneous coronary intervention (PCI) of the left anterior descending artery (LAD) with a drug-eluting stent (HREF). He also has chronic kidney disease (CKD), a history of cerebrovascular accident (CVA), and atrial fibrillation (AF) for which he is currently taking a non-vitamin K antagonist oral anticoagulant (NOAC) rivaroxaban. His current medications include bicor, aspirin, statin, rivaroxaban, and entresto. 

On examination, his body mass index (BMI) is within normal limits, and his electrocardiogram (ECG) shows a normal sinus rhythm (NSR) with a heart rate of 80 beats per minute. His blood pressure is 120/75 mmHg, and his jugular venous pressure

### Step 3: Analyse the Clinical Note to check if the doctor has missed anything?

- **Input**: Medical Shorthand (`clinical_note`)
- **Output**: Clinical Note (`feedback`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant.

Users will paste in a clinical note. You will respond by analysing the clinical note and checking if the doctor has missed anything.
<|im_end|>
<|im_start|>user
******THE-LONG-FORM-CLINICAL-NOTE-GOES-HERE******
<|im_end|>
<|im_start|>assistant
```


In [53]:
prompt_step_3 = """
<|im_start|>system
You are an medical assistant. 

Users will paste in a clinical note. You will respond by analysing the clinical note and checking if the doctor has missed anything.
<|im_end|>
<|im_start|>user
""" + clinical_note + """
<|im_end|>
<|im_start|>assistant
Clinical Note:
"""

completion_step_3 = openai.Completion.create(
    deployment_id=deployment_name,
    prompt=prompt_step_3 , 
    stop="<|im_end|>", 
    temperature=0.3,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0)

feedback = completion_step_3['choices'][0]['text']

print("Feedback:")
print(feedback)

Feedback:
- Patient is a 73-year-old male
- Complaints of chronic progressive onset of exertional dyspnea (CPOE) and shortness of breath on exertion (SOBOE) for the past 3 months
- PMHx includes a ST-elevation myocardial infarction (STEMI) in 2002, which was treated with percutaneous coronary intervention (PCI) of the left anterior descending artery (LAD) with a drug-eluting stent (HREF)
- Chronic kidney disease (CKD), a history of cerebrovascular accident (CVA), and atrial fibrillation (AF) for which he is currently taking a non-vitamin K antagonist oral anticoagulant (NOAC) rivaroxaban
- Current medications include bicor, aspirin, statin, rivaroxaban, and entresto
- On examination, BMI is within normal limits, ECG shows a normal sinus rhythm (NSR) with a heart rate of 80 beats per minute, blood pressure is 120/75 mmHg, JVP is not raised, no pedal edema, and lungs are clear on auscultation
- Plan is to focus on lifestyle modifications, including diet, and to follow up with his general